#### (1) Data collection using both, Web Scraping and a Web API.
#### (2) Data preparation (e.g. remove missing values and duplicates, create new variables, enrich the data with open data).
#### (3) Data storage in a database like sqlite or MySQL.
#### (4) Non-graphical and graphical exploratory data analysis (EDA).
#### (5) Use of either regression or classification as the modelling method.
#### (6) Model evaluation using suitable measures of fit.
#### (7) Correct interpretation of model results and measures of fit.
#### (8) Making the material (data, Jupyter notebooks, ...) available on Moodle.

In [ ]:
import
